In [1]:
# Load the aiida profile
from aiida import load_profile
aiida_profile = load_profile()
aiida_profile.name

'default'

In [2]:
## Loading the some require packages and module
from aiida_kkr.workflows import (combine_imps_wc, kkr_flex_wc,
                                 kkr_imp_wc, kkr_imp_sub_wc
                                )
from aiida_kkr.calculations import KkrCalculation, KkrimpCalculation
from aiida.orm import (Group, load_group, load_node, Dict, Code
                      )
from aiida_kkr.tools import kkrparams
from aiida.engine import submit

In [3]:
# One very frequently used option
metadata_option_1 = {'max_wallclock_seconds': 36000,'resources': 
               {'tot_num_mpiprocs': 48, 'num_machines': 1},
              'custom_scheduler_commands': 
              '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
              'withmpi': True
                    }

In [4]:
# Note that here for doped 3d and 4d atoms into the Bi2Te3 host saved in group-74

group_74 = load_group(74)
## first node of group-74 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files

imp_wc_with_gf = load_node(16100)
imp1_wc = group_74.nodes[1]
imp2_wc = group_74.nodes[2]

# gf_host_remote = imp_wc_with_gf.inputs.remote_data_gf
# host_gf_wc = imp_wc_with_gf.get_outgoing(node_class=kkr_flex_wc).all()[0].node


impurity1_output_node = imp1_wc.outputs.workflow_info
impurity2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':1}

In [5]:
# group_74 = load_group(91)
# node_list = list(group_74.nodes)
# i=0
# for node in node_list:
#     i += 1
#     impurity_info = node.inputs.impurity_info.get_dict()
#     print(impurity_info)
# print(i)

In [6]:
# kkr_code_iffslurm = kkrflex@iffslurm

In [7]:
scf_node = imp1_wc.get_outgoing(node_class=kkr_imp_sub_wc).all()[0].node
## kkr_flex_wc
host_gf = imp1_wc.inputs.remote_data_gf.get_incoming(node_class=kkr_flex_wc).all()[0].node
kkr_calc = host_gf.get_outgoing(node_class=KkrCalculation).all()[0].node
kkr_flex_kkr_param = kkr_calc.inputs.parameters

In [8]:
kkr_flex_kkr_param = kkr_calc.inputs.parameters.get_dict()
kkr_flex_kkr_param['RCLUSTZ'] *= 1
kkr_flex_kkr_param['NSHELD'] = 2500

In [9]:
# kkr_flex_kkr_param

In [10]:
settings = combine_imps_wc.get_wf_defaults()
builder = combine_imps_wc.get_builder()
settings['jij_run']= True
settings['retrieve_kkrflex'] = False
settings['mag_init']= False
settings['threshold_switch_high_accuracy'] = 1e-2
settings['convergence_criterion'] = 1e-7 #convergence_criterion
settings['threshold_aggressive_mixing']= 0.02
settings['strmix']= 0.01
settings['aggrmix'] = 0.05
settings['broyden-number'] = 20
settings['nsteps'] = 100
settings['kkr_runmax'] = 10

Version of workflow: 0.3.1


In [11]:
options = scf_node.inputs.options.get_dict()
options['max_wallclock_seconds'] = 36000*1
# options

In [12]:
builder.impurity1_output_node = impurity1_output_node
builder.impurity2_output_node = impurity2_output_node
builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
builder.scf.kkrimp = scf_node.inputs.kkrimp
builder.scf.options = Dict(dict=options)
builder.scf.wf_parameters = scf_node.inputs.wf_parameters

# host_gf namespace setup
builder.host_gf.kkr = host_gf.inputs.kkr
builder.host_gf.options = host_gf.inputs.options
builder.host_gf.params_kkr_overwrite = Dict(dict=kkr_flex_kkr_param) #host_gf.inputs.wf_parameters

builder.wf_parameters_overwrite = Dict(dict=settings)
builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)

In [13]:
# To submit the builder
# combined_imp_submission = submit(builder)

In [17]:
# Note that here for doped 3d atoms into the Bi2Te3 host

group_74 = load_group(74)
## first node of group-64 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files

imp_wc_with_gf = load_node(16100)
imp1_wc = group_74.nodes[1]
imp2_wc = group_74.nodes[2]

scf = imp1_wc

host_wf_parameters = Dict(dict={'NSHELD' : 2500})
host_kkr = Code().get_from_string('kkr@claix18_init')

scf_kkrimp = Code().get_from_string('kkrflex@claix18_init')
options = Dict(dict=metadata_option_1)

imp1_output_node = imp1_wc.outputs.workflow_info
imp2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':1}

settings = combine_imps_wc.get_wf_defaults()
settings['jij_run'] = True
settings['dos_run'] = False
settings['lmdos'] = False
settings['strmix'] = 0.01
settings['aggrmix'] = 0.01
settings['threshold_aggressive_mixing'] = 0.01
settings['convergence_criterion'] = 1e-7
settings['threshold_switch_high_accuracy'] = 1e-2
settings['retrieve_kkrflex'] = False
settings['mag_init'] = False
settings['mixreduce'] = 0.5
settings['nsteps'] = 100
settings['kkr_runmax'] = 10
settings['broyden-number'] = 20




combine_imp_builder = combine_imps_wc.get_builder()
combine_imp_builder.impurity1_output_node = imp1_output_node
combine_imp_builder.impurity2_output_node = imp2_output_node
combine_imp_builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
combine_imp_builder.scf.kkrimp = scf_kkrimp
combine_imp_builder.scf.options = options
combine_imp_builder.scf.wf_parameters = scf.inputs.wf_parameters

# combine_imp_builder.gf_host_remote = 
# host_gf namespace setup
combine_imp_builder.host_gf.kkr = host_kkr
combine_imp_builder.host_gf.options = options
combine_imp_builder.host_gf.params_kkr_overwrite = host_wf_parameters #host_gf.inputs.wf_parameters

combine_imp_builder.wf_parameters_overwrite = Dict(dict=settings)
combine_imp_builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)

# submission = submit(combine_imp_builder)

Version of workflow: 0.3.1


In [184]:
## Here trying to work with host Au and other impurity
## THe calc is taken from the wasmer
imp_Au_group = load_group(85)
wc_list = list(imp_Au_group.nodes)


imp_wc_1 = wc_list[0]
# print('first imp wc : ', imp_wc_1.label)
imp_wc_2 = wc_list[1]
# print('econd imp wc : ', imp_wc_2.label)
# setting the scf and host namespace

scf = imp_wc_1
host_wf_parameters = Dict(dict={'NSHELD' : 2500,
                                'BZDIVIDE': [40, 40, 40],
                                'FCM': 20,
                                'EMAX': 1.2,
                                'EMIN': -0.8,
                                'GMAX': 65.0,
                                'LMAX': 3,
                                'NCHEB': 12,
                              
                               })

host_kkr = Code().get_from_string('kkr@claix18_init')
scf_kkrimp = Code().get_from_string('kkrflex@claix18_init')
options = Dict(dict=metadata_option_1)

imp1_output_node = imp_wc_1.outputs.workflow_info
imp2_output_node = imp_wc_2.outputs.workflow_info
offset_imp2 = {'index':1}

settings = combine_imps_wc.get_wf_defaults()
settings['jij_run'] = False
settings['dos_run'] = False
settings['lmdos'] = False
settings['strmix'] = 0.001
settings['aggrmix'] = 0.008
settings['threshold_aggressive_mixing'] = 0.008
settings['convergence_criterion'] = 1e-7
settings['threshold_switch_high_accuracy'] = 1e-2
settings['retrieve_kkrflex'] = False
settings['mag_init'] = False
settings['mixreduce'] = 0.5
settings['nsteps'] = 100
settings['kkr_runmax'] = 10

combine_imp_builder = combine_imps_wc.get_builder()
combine_imp_builder.impurity1_output_node = imp1_output_node
combine_imp_builder.impurity2_output_node = imp2_output_node
combine_imp_builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
combine_imp_builder.scf.kkrimp = scf_kkrimp
combine_imp_builder.scf.options = options
combine_imp_builder.scf.wf_parameters = scf.inputs.wf_parameters

# host_gf namespace setup
combine_imp_builder.host_gf.kkr = host_kkr
combine_imp_builder.host_gf.options = options
combine_imp_builder.host_gf.params_kkr_overwrite = host_wf_parameters #host_gf.inputs.wf_parameters

combine_imp_builder.wf_parameters_overwrite = Dict(dict=settings)
combine_imp_builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)
# submission = submit(combine_imp_builder)

Version of workflow: 0.3.1


In [29]:
# An attemp to run a bunch of wc of the double impurity from two groups of single impurity wc or 
# two list of the single impurity wc

def submit_double_imp_wc_bunch( si_imp_list_1, si_imp_list_2, kkr_code, kkr_imp_code,
                                builder_options,  group_label, group_descr,
                                offset_imp2={'index':1}, max_submission=20, setting=None, 
                                scf_wf_parameter=None, host_gf_wf_parameters=None, 
                              
                              ):
    """
    params: 
        : Two groups : Two single imp wc groups.
        : Two lists : Two single imp wc list.
        : Settings Dict : Parameter settings.
        : Builder_options Dict : builder_option
        : group label str : Create a group by this name if it does not exist in the DB.
        : group descr str : Set the created group description either it exists or not, it not a new group
                            will be created.
    """
    
    # Create group or load existing group
    group = group_not_exist_create(group_label=group_label, group_descr=group_descr)
    
#     test_data and prepare for further
    if not isinstance(si_imp_list_1, list):
        print('The given impurity wc list 1 is not the list type.')

    if not isinstance(si_imp_list_2, list):
        print('The given impurity wc list 2 is not the list type.')
    
    all_combination_dict = {}
    all_success_dict = {}
    all_submission_dict = {}
    all_resedue_dict = {}
    
    tot_wc_num = 0
    all_submission_num = 0
    all_success_num = 0
    all_failed_num = 0
    
    # Here to create all the possible dict
    for i in range(len(si_imp_list_1)):
        for j in range(len(si_imp_list_2)):
            node_truple = (i, j)
            pk_truple = (i.pk, j.pk)
            imp1_info = i.inputs.impurity_info.get_dict()
            imp2_info = j.inputs.impurity_info.get_dict()
            ilayer1 = str(imp1_info.['ilayer_center'])
            ilayer2 = str(imp2_info.['ilayer_center'])
            
            label= i.label.split(':')[0] + ':'+ j.label+':'+ilayer1+':'+ilayer2        
            all_combination_dict[tot_wc_num]['node_truple'] = node_truple
            all_combination_dict[tot_wc_num]['pk_truple'] = pk_truple
            all_combination_dict[tot_wc_num]['label'] = label
            tot_wc_num +=1
    
    all_resedue_dict = all_combination_dict.copy()
    
    # To create the submission process
    for key, val in all_combination_dict.items():
        # ------------
        imp1_wc_node = all_combination_dict[key]['node_truple'][0]
        imp2_wc_node = all_combination_dict[key]['node_truple'][1]
        label = all_combination_dict[key]['label']

        if all_failed_num == 10: # TODO: add a argumen in the function
            break

        
        if debug: # TODO add the debug in the function arguments
            print('imp1_pk for submission', imp1_wc_node.pk)
            print('imp2_pk for submission', imp2_wc_node.pk)
        #TODO: Check is the wc is in already Group 
        # if yes 'continue'

        if not dry_run: # TODO add the dry run in the function argument
            submission = submit_combine_imp(imp1_wc_node, imp2_wc_node, kkr_code, kkr_imp_code, 
                                        offset_imp2, options, scf_wf_parameter, label, 
                                        host_gf_wf_parameters, settings)
            all_submission_dict[key] = all_combination_dict[key].copy()
            
        all_submission_num += 1
        while(all_submission_num - all_success_num + all_failed_num >= max_submission) or 
                (tot_wc_num - all_submission_num == 0):
                if submission.is_finished = True:
                    if submission.exit_statu = 0 : # TODO: Check here is the submission have right variable
                        all_success_num +=1
                        all_success_dict[key] = all_combination_dict[key].copy()
                        all_resedue_dict.pop(key)
                        ## TODO: Add in to the group
                    else:
                        all_failed_num += 1
                        if all_failed_num == 10: # TODO: add a argumen in the function
                            break
        
            
        
        
        
    
    
            

SyntaxError: invalid syntax (<ipython-input-29-8d7ac36d7e5a>, line 34)

In [26]:
def submit_combine_imp(imp1_wc_node, imp2_wc_node, kkr_code, kkr_imp_code, offset_imp2, options, 
                       scf_wf_parameter, label, host_gf_wf_parameters, settings):
    
    from aiida_kkr.workflows import combine_imps_wc, kkr_imp_sub_wc, kkr_flex_wc

    imp1_output = imp1_wc_node.outputs.workflow_info
    imp2_output = imp2_wc_node.outputs.workflow_info
    if scf_wf_parameter==None:
        sub_wc1_node = imp1_wc_node.get_outgoing(node_class=kkr_imp_sub_wc).first().node
        scf_wf_parameters = sub_wc1_node.inputs.wf_parameters # TODO : check  'inputs.wf_parameters' is right
    if params_kkr_overwrite==None:
        sub_gf_write_out = imp1_wc_node.get_outgoing(node_class=kkr_flex_wc).first().node
        host_gf_wf_parameters = sub_gf_write_out.inputs.params_kkr_overwrite
    if settings==None:
        settings = imp1_wc_node.inputs.wf_parameters_overwrite
    if label==None:
        label = 'pk' + str(imp1_wc_node.pk)+':'+ str(imp1_wc_node.pk)
    
    builder = combine_imps_wc.get_builder()
    builder.impurity1_output_node = imp1_output
    builder.impurity2_output_node = imp2_output
    builder.offset_imp2 = offset_imp2
    
    builder.scf.kkrimp = kkr_imp_code
    builder.scf.options = options
    builder.scf.wf_parameters = scf_wf_parameters
    
    builder.host_gf.kkr = kkr_code
    builder.host_gf.options = options
    builder.host_gf.params_kkr_overwrite = host_gf_wf_parameters #host_gf.inputs.wf_parameters
    if settings!=None:
        builder.wf_parameters_overwrite = settings
    builder.metadata.label = label
#     submission = submit(builder)
    return submission
def group_not_exist_create(group_label, group_descr):
    from aiida.orm import load_group, Group
    """
        Check the group exist either must create
    """
    
    try:
         group = load_group(group_label)
    except:
        print('Group named {} is not exist but is being created .'.format(group_label))
        group = Group(label=group_label, description=group_descr)
        group.store()
        print('Newly created group pk {}'.format(group.pk))
    return group

# TODO:
# Define a function to check any wc with the similar label is exist in the  work_chain_group
# if exist return true
def check_wc_exist_in_group(group, wc_labe):
    

In [28]:
group_label = 'test_group_delete_this'
group_descr = 'Test_group_descr'
# group_label.is_stored

group_label=None
group_descr=None

group = group_not_exist_create(group_label=group_label, group_descr=group_descr)
group.is_stored

Group named None is not exist but is being created .


ValueError: Group label must be provided